In [19]:
import os
import shutil

import requests

import time
import numpy as np

import pandas as pd
import polars as pl
import polars.selectors as cs


import duckdb

import pyarrow.parquet as pq

In [21]:
os.environ["POLARS_TEMP_DIR"] = "D:\\TEMP_DIR"
os.environ["TMPDIR"] = "D:\\TEMP_DIR"
os.environ["TEMP"] = "D:\\TEMP_DIR"
os.environ["TMP"] = "D:\\TEMP_DIR"

duckdb.sql("""
    SET temp_directory = "D:\\TEMP_DIR"
""")

In [3]:
import os

# Verificación de la variable de entorno
temp_path = os.environ.get("POLARS_TEMP_DIR")

if temp_path:
    print(f"✅ Polars usará: {temp_path}")
    if not os.path.exists(temp_path):
        print("⚠️ Advertencia: La carpeta no existe, Polars podría fallar o usar el default.")
else:
    print("❌ La variable no está seteada. Polars usará el disco C (Temp del sistema).")

✅ Polars usará: D:\TEMP_DIR


## <p style='text-align: center; text-decoration: underline; color: #10A0B4;'> Datos oficiales de la **Union Europea** </p>
[Origen de los datos](https://www.eea.europa.eu/en/datahub/datahubitem-view/fa8b1229-3db6-495d-b18e-9c9b3267c02b)

---

## Autos registrados desde **2010** hasta **2023**

---

#### El dataset contiene mas de **16 Gigas** de informacion (80.378.486 filas).

El gran tamano del conjunto de datos trae problemas:
- No puedo subirlo a GitHub (+100MB). 
- Lento procesamiento.

Vamos a resolver ambas cuentiones achicando el dataset de manera tal que el impacto por usar menos informacion sea minimo

---

El archivo pesa +16GB y tiene más de 80 millones de filas. Tarda +3 min en ejecutar. 

In [4]:
url = 'ignore_data.csv'
file_weight = os.path.getsize(url)
print('Database size: ', round((file_weight/1024)/1024/1024, 2), 'GB')

count = duckdb.sql("SELECT count(*) FROM ignore_data.csv").fetchone()[0]
print(f"El archivo .csv tiene {count} registros.")

duckdb.sql("SELECT * FROM 'ignore_data.csv' LIMIT 0").show() # 10 seg aprox. 
duckdb.sql("SUMMARIZE ignore_data.csv").show(max_rows=100) # 10 seg aprox. 
# pl.scan_parquet("ignore_data_complete_clean.parquet").describe().head(45) # 45 seg aprox.

Database size:  16.38 GB


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

El archivo .csv tiene 80378486 registros.
┌───────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬───────┬────────┬───────┬──────────────┬──────────────┬────────┬──────────┬──────────┬─────────┬─────────┬──────────┬─────────┬───────────┬─────────┬───────────────┬───────────────┬────────┬───────┬─────────┬───────┬──────────────────────┬──────────────────┬─────────┬─────────┬─────────────────────┐
│  ID   │ Country │   VFN   │   Mp    │   Mh    │   Man   │   MMS   │   Tan   │    T    │   Va    │   Ve    │   Mk    │   Cn    │   Ct    │   Cr    │   r   │ m (kg) │  Mt   │ Enedc (g/km) │ Ewltp (g/km) │ W (mm) │ At1 (mm) │ At2 (mm) │   Ft    │   Fm    │ ec (cm3) │ ep (KW) │ z (Wh/km) │   IT    │ Ernedc (g/km) │ Erwltp (g/km) │   De   │  Vf   │ Status  │ year  │ Date of registration │ Fuel consumption │   ech   │  RLFI   │ Electric range (km) │
│ int64 │ varchar │ varchar │ varchar │ varchar │ varchar 

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌──────────────────────┬─────────────┬────────────────────────────────────────────────────┬────────────────────────────────────────────────────────────────────┬───────────────┬──────────────────────┬──────────────────────┬───────────────────────┬──────────────────────┬─────────────────────┬──────────┬─────────────────┐
│     column_name      │ column_type │                        min                         │                                max                                 │ approx_unique │         avg          │         std          │          q25          │         q50          │         q75         │  count   │ null_percentage │
│       varchar        │   varchar   │                      varchar                       │                              varchar                               │     int64     │       varchar        │       varchar        │        varchar        │       varchar        │       varchar       │  int64   │  decimal(9,2)   │
├──────────────────────┼─────────────

Tabla informacional de códigos y datatypes

<table>
<tbody>
<tr>
    <td>Name</td>
    <td>Definition</td>
    <td>Datatype </td>

</tr>
<tr>
    <td>ID</td>
    <td>Identification number.</td>
    <td>integer </td>
</tr>
<tr>
    <td>MS</td>
    <td>Member state.</td>
    <td>varchar(2) </td>
</tr>
<tr>
    <td>Mp</td>
    <td>Manufacturer pooling.</td>
    <td>varchar(50) </td>
</tr>
<tr>
    <td>VFN</td>
    <td>Vehicle family identification number.</td>
    <td>varchar(25) </td>
</tr>
<tr>
    <td>Mh</td>
    <td>Manufacturer name EU standard denomination .</td>
    <td>varchar(50) </td>
</tr>
<tr>
    <td>Man</td>
    <td>Manufacturer name OEM declaration.</td>
    <td>varchar(50) </td>
</tr>
<tr>
    <td>MMS</td>
    <td>Manufacturer name MS registry denomination .</td>
    <td>varchar(125) </td>
</tr>
<tr>
    <td>TAN</td>
    <td>Type approval number.</td>
    <td>varchar(50) </td>
</tr>
<tr>
    <td>T</td>
    <td>Type.</td>
    <td>varchar(25) </td>
</tr>
<tr>
    <td>Va</td>
    <td>Variant.</td>
    <td>varchar(25) </td>
</tr>
<tr>
    <td>Ve</td>
    <td>Version.</td>
    <td>varchar(35) </td>
</tr>
<tr>
    <td>Mk</td>
    <td>Make.</td>
    <td>varchar(25) </td>
</tr>
<tr>
    <td>Cn</td>
    <td>Commercial name.</td>
    <td>varchar(50) </td>
</tr>
<tr>
    <td>Ct</td>
    <td>Category of the vehicle type approved.</td>
    <td>varchar(5)  </td>
</tr>
<tr>
    <td>Cr</td>
    <td>Category of the vehicle registered.</td>
    <td>varchar(5)  </td>
</tr>
<tr>
    <td>M (kg)</td>
    <td>Mass in running order Completed/complete vehicle .</td>
    <td>integer </td>
</tr>
<tr>
    <td>Mt</td>
    <td>WLTP test mass.</td>
    <td>integer</td>
</tr>
<tr>
    <td>Enedc (g/km)</td>
    <td>Specific CO2 Emissions (NEDC).</td>
    <td>integer </td>
</tr>
<tr>
    <td>Ewltp (g/km)</td>
    <td>Specific CO2 Emissions (WLTP).</td>
    <td>integer </td>
</tr>
<tr>
    <td>W (mm)</td>
    <td>Wheel Base.</td>
    <td>integer </td>
</tr>
<tr>
    <td>At1 (mm)</td>
    <td>Axle width steering axle.</td>
    <td>integer </td>
</tr>
<tr>
    <td>At2 (mm)</td>
    <td>Axle width other axle.</td>
    <td>integer </td>
</tr>
<tr>
    <td>Ft</td>
    <td>Fuel type.</td>
    <td>varchar(25) </td>
</tr>
<tr>
    <td>Fm</td>
    <td>Fuel mode.</td>
    <td>varchar(1)  </td>
</tr>
<tr>
    <td>Ec (cm3)</td>
    <td>Engine capacity.</td>
    <td>integer </td>
</tr>
<tr>
    <td>Ep (KW)</td>
    <td>Engine power.</td>
    <td>integer </td>
</tr>
<tr>
    <td>Z (Wh/km)</td>
    <td>Electric energy consumption.</td>
    <td>integer </td>
</tr>
<tr>
    <td>IT</td>
    <td>Innovative technology or group of innovative technologies.</td>
    <td>varchar(25) </td>
</tr>
<tr>
    <td>Ernedc (g/km)</td>
    <td>Emissions reduction through innovative technologies.</td>
    <td>float </td>
</tr>
<tr>
    <td>Erwltp (g/km)</td>
    <td>Emissions reduction through innovative technologies (WLTP).</td>
    <td>float </td>
</tr>
<tr>
    <td>De</td>
    <td>Deviation factor.</td>
    <td>float </td>
</tr>
<tr>
    <td>Vf</td>
    <td>Verification factor.</td>
    <td>integer </td>
</tr>
<tr>
    <td>R</td>
    <td>Total new registrations.</td>
    <td>integer </td>
</tr>
<tr>
    <td>Year</td>
    <td>Reporting year.</td>
    <td>integer </td>
</tr>
<tr>
    <td>Status</td>
    <td>P = Provisional data, F = Final data.</td>
    <td>varchar(1)  </td>
</tr>
<tr>
    <td>Version_file</td>
    <td>Internal versioning of deliverables.</td>
    <td>varchar(10) </td>
</tr>
<tr>
    <td>E (g/km)</td>
    <td>Specific CO2 Emission. Deprecated value, only relevant for data until 2016.</td>
    <td>float </td>
</tr>
<tr>
    <td>Er (g/km)</td>
    <td>Emissions reduction through innovative technologies. Deprecated value, only relevant for data until 2016.</td>
    <td>float </td>
</tr>
<tr>
    <td>Zr</td>
    <td>Electric range.</td>
    <td>integer </td>
</tr>
<tr>
    <td>Dr</td>
    <td>Registration date.</td>
    <td>date </td>
</tr>
<tr>
    <td>Fc</td>
    <td>Fuel consumption.</td>
    <td>float </td>
</tr>
</tbody>
</table>

---

<p style="font-size: 35px; text-align: center; color: yellow;"> Limpiado de datos </p>

<p style="font-size: 25px;">Crearé el DataFrame con la información cruda. Tal como la descargamos.</p>
<p style="font-size: 20px">Guardaré el DataFrame en archivo .parquet ya que internamente divide la información en bloques para que a la hora de procesarla vaya de a bloques de 1M registros por ejemplo.</p>

<p style="font-size: 25px">Para poder procesar los 80 millones de registros, usar <u>pandas</u> es inviable para mi computadora de 16GB de RAM. Provoca error 'MemoryError'</p>

- Pandas trabaja con el DF en memoria y a demás el DF debe de ocupar como máximo entre el 10% y 30% de la memoria RAM. 


<p>Se puede guardar el DataFrame en 80 archivos .parquet con 1M de registros cada uno o <u>en uno único</u> de 80M de registros. Usar <u>polars</u> o <u>duckdb</u> para procesar la información es mejor ya que ambos procesan archivos .parquet de a bloques (cargando y descargando en RAM los bloques .parquet) y a demás utilizan todos los núcleos de la CPU. </p>

- <b><u>Polars:</u></b> 
    - <b>Procesamiento:</b> Utiliza un motor escrito en Rust que procesa datos de forma paralela
    - <b>Sintaxis:</b> Utiliza expresiones parecidas a las de pandas pero más optimizadas. 
    - <b>Memoria:</b> Trabaja con "streaming" ("Lazy"), cargando y transformando sólolos trozos necesarios en cada momento. 

- <b><u>duckDB:</u></b>
    - <b>Procesamiento:</b> Es un motor SQL analítico que puede ejecutarse directamente en el script de Python.
    - <b>Flexibilidad:</b> Permite realizar transformaciones complejas mediante consultas SQL o su propia API de Python, siendo extremadamente eficiente para cruzar (join) múltiples archivos grandes.
    - <b>Out-of-Core:</b> Si los datos superan la RAM, DuckDB puede usar el disco duro como apoyo para que el proceso no falle.

<p style="font-size: 35px; text-align: center; color: orange;">Usaré las 2 librerías. POLARS para modificaciones del DF y DUCKDB para consultar.</p>

##### Si el registro no tiene información en: Marca, modelo, tipo de combustible y ano de fabricación entonces no me sirve.  

In [28]:
columns_to_load = ['ID', 'year', 'Country', 'Mh', 'z (Wh/km)', 'ep (KW)', 'Mk', 'Cn', 
                   'Electric range (km)', 'm (kg)', 'Ewltp (g/km)', 'Ft', 'ec (cm3)', 
                   'Fuel consumption ', 'MMS']

rename_columns = {
    'm (kg)': 'mass_kg', 'Ewltp (g/km)': 'co2_emission_g/km', 'Ft': 'fuel_type',
    'ec (cm3)': 'engine_size_cm3', 'Fuel consumption ': 'fuel_consumption_l/100km',
    'year': 'year_of_fabrication', 'Mh': 'manufacturer', 'z (Wh/km)': 'energy_consumption_Wh/km',
    'ep (KW)': 'electric_power_KW', 'Mk': 'model', 'Cn': 'commercial_name',
    'Electric range (km)': 'electric_range_km', 'MMS': 'manufacturar_name', "Country": "country"
}

# scan_csv (Lazy) - Esto NO carga el archivo en RAM
df_lazy = (
    pl.scan_csv('ignore_data.csv', infer_schema_length=10000) # En base a las primeras 10k filas, Polars infiere el tipo de cada columna.
    .select(columns_to_load)
    .rename({k: v for k, v in rename_columns.items() if k in columns_to_load})
    .drop_nulls(subset=['model', 'commercial_name', 'fuel_type', 'year_of_fabrication'])
    .with_columns(cs.string().str.strip_chars().str.to_lowercase().replace("", None))
)

print("Iniciando procesamiento en streaming...")
df_lazy.sink_parquet(f"ignore_data_complete_clean.parquet") # 2 min aprox. para procesar. 
print("Done!")

file_weight = os.path.getsize("ignore_data_complete_clean.parquet")
print('.parquet size: ', round((file_weight/1024)/1024/1024, 2), 'GB')

Iniciando procesamiento en streaming...
Done!
.parquet size:  0.4 GB


In [6]:
duckdb.sql("SELECT * FROM 'ignore_data_complete_clean.parquet' LIMIT 5").show()

┌────────┬─────────────────────┬─────────┬────────────────────────────┬──────────────────────────┬───────────────────┬───────────────┬──────────────────────────┬───────────────────┬─────────┬───────────────────┬───────────┬─────────────────┬──────────────────────────┬───────────────────┐
│   ID   │ year_of_fabrication │ country │        manufacturer        │ energy_consumption_Wh/km │ electric_power_KW │     model     │     commercial_name      │ electric_range_km │ mass_kg │ co2_emission_g/km │ fuel_type │ engine_size_cm3 │ fuel_consumption_l/100km │ manufacturar_name │
│ int64  │        int64        │ varchar │          varchar           │          int64           │       int64       │    varchar    │         varchar          │      varchar      │  int64  │       int64       │  varchar  │      int64      │         varchar          │      varchar      │
├────────┼─────────────────────┼─────────┼────────────────────────────┼──────────────────────────┼───────────────────┼───────────────

#### Edit datatype for reduce file size and some others changes. 

In [30]:
# read_parquet() vs sink_parquet() - La primera carga el archivo en memoria, mientras que la segunda lo hace en streaming (carga de a pedzos en memoria y luego pasa a disco). Si el archivo me entra en memoria, es más rápido cargarlo con read_parquet() y hacer las transformaciones en memoria ya que escribir en disco es más lento. 

# df = pl.read_parquet("ignore_data_complete_clean.parquet")

# df = df.with_columns((pl.col("engine_size_cm3")/1000)
#     .round(2)
#     .alias("engine_size_L")
# ).drop("engine_size_cm3")

# df = df.with_columns([
#     pl.col(["country", "manufacturer", "model", "commercial_name", "fuel_type", "manufacturar_name"]).cast(pl.Categorical), # Categorical: Es un tipo de dato que almacena los valores únicos de una columna y luego asigna un código numérico a cada valor. Esto reduce el espacio de almacenamiento y mejora el rendimiento en operaciones de filtrado y agrupamiento.
#     pl.col("ID").cast(pl.Int32), 
#     pl.col(["year_of_fabrication", "energy_consumption_Wh/km", "electric_power_KW", "electric_range_km", "mass_kg", "co2_emission_g/km"]).cast(pl.Int16), 
#     pl.col(["fuel_consumption_l/100km", "engine_size_L"]).cast(pl.Float32)
# ])

#df_optimized.write_parquet("ignore_data_optimized_clean.parquet", compression="zstd", compression_level=15)


input_path = "ignore_data_complete_clean.parquet"
temp_path = "temporal_ignore_data_complete_clean.parquet"

query = (
    pl.scan_parquet(input_path)
    .with_columns(
        (pl.col("engine_size_cm3") / 1000).round(2).alias("engine_size_L")
    )
    .drop("engine_size_cm3") # Eliminamos la columna original para que no esté duplicada
    .with_columns([
        pl.col("ID").cast(pl.Int32), 
        pl.col("year_of_fabrication").cast(pl.String),
        pl.col(["energy_consumption_Wh/km", "electric_power_KW", "electric_range_km", "mass_kg", "co2_emission_g/km"]).cast(pl.Int16), 
        pl.col(["fuel_consumption_l/100km", "engine_size_L"]).cast(pl.Float32), 
        pl.col(["country", "manufacturer", "model", "commercial_name", "fuel_type", "manufacturar_name"]).cast(pl.Categorical) # Categorical: Es un tipo de dato que almacena los valores únicos de una columna y luego asigna un código numérico a cada valor. Esto reduce el espacio de almacenamiento y mejora el rendimiento en operaciones de filtrado y agrupamiento.
    ])
)

# Sirve para que no me cargue en memoria el DataFrame. Pero ya que el .parquet es mas liviano, lo puedo cargar en memoria y hacer las transformaciones. En memoria es más rápido que escribir en disco. 
query.sink_parquet(temp_path, compression="zstd", compression_level=15) 
shutil.copy2(temp_path, "ignore_backup_1.parquet")

os.remove(input_path) 
os.rename(temp_path, input_path)

file_weight = os.path.getsize(input_path)
print('.parquet size: ', round((file_weight/1024)/1024, 2), 'MB')
print("Done!")

.parquet size:  427.99 MB
Done!


In [31]:
file_weight = os.path.getsize("ignore_data_complete_clean.parquet")
print('.parquet size: ', round((file_weight/1024)/1024/1024, 2), 'GB')
duckdb.sql("SUMMARIZE SELECT * FROM 'ignore_data_complete_clean.parquet'").show(max_rows=25)

.parquet size:  0.42 GB


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌──────────────────────────┬─────────────┬────────────────────────┬────────────────────────┬───────────────┬────────────────────┬────────────────────┬────────────────────┬───────────────────┬────────────────────┬──────────┬─────────────────┐
│       column_name        │ column_type │          min           │          max           │ approx_unique │        avg         │        std         │        q25         │        q50        │        q75         │  count   │ null_percentage │
│         varchar          │   varchar   │        varchar         │        varchar         │     int64     │      varchar       │      varchar       │      varchar       │      varchar      │      varchar       │  int64   │  decimal(9,2)   │
├──────────────────────────┼─────────────┼────────────────────────┼────────────────────────┼───────────────┼────────────────────┼────────────────────┼────────────────────┼───────────────────┼────────────────────┼──────────┼─────────────────┤
│ ID                       │ INT

#### El auto que menos consume en el mundo: 
    - Diesel es el Renault Clio Blue dCi 100 con un consumo de 3,6l/100km. 
    - Petrol, es el Suzuki Swift 1.2 con 4,4l/100km.  
#### El auto más pesado y liviano del mundo: 
    - Más pesado es el Rolls-Royce Specrte con 2975kg
    - Más liviano es el suzuki alto (versíon Japonesa) con 810kg

In [32]:
duckdb.sql("SELECT COUNT(*) FROM 'ignore_data_complete_clean.parquet'").show()
duckdb.sql("""
    SELECT COUNT(CASE WHEN ("fuel_consumption_l/100km" < 3.6 AND "fuel_type" IN ('petrol', 'diesel')) OR ("mass_kg" <= 800 OR "mass_kg" >= 3000) THEN 1 END) AS eliminar
    FROM 'ignore_data_complete_clean.parquet'
""").show()

######## Si tengo más de 32GB de RAM, puedo cargar el .parquet en memoria y hacer las transformaciones sin necesidad de escribir en disco. 
# Eliminamos los registros que no tienen sentido. 
# df = pl.read_parquet("ignore_data_complete_clean.parquet")
# df = df.filter(
#     (
#         # Condition to remove. 
#         (pl.col("fuel_consumption_l/100km") < 3.6) & (pl.col("fuel_type").is_in(["petrol", "diesel"])) 
#         |
#         (pl.col("mass_kg") <= 800) | (pl.col("mass_kg") >= 3000)
#     ).not_().fill_null(True)
# )   
# duckdb.sql("SELECT COUNT(*) FROM df ").show()
# df.write_parquet("ignore_data_complete_clean.parquet")
# duckdb.sql("SELECT COUNT(*) FROM 'ignore_data_complete_clean.parquet'").show()

input_path = "ignore_data_complete_clean.parquet"
temp_path = "temporal_ignore_data_complete_clean.parquet"

query = (
    pl.scan_parquet(input_path)
    .filter(
        # Condition to remove. 
        (
            ((pl.col("fuel_consumption_l/100km") < 3.6) & (pl.col("fuel_type").is_in(["petrol", "diesel"]))) 
            | 
            ((pl.col("mass_kg") <= 800) | (pl.col("mass_kg") >= 3000))
        ).eq(False).fill_null(True)
    )
)

df_result_count = query.select(pl.len()).collect()
print("Cantidad de registros después de la limpieza: ", df_result_count[0, 0])
query.sink_parquet(temp_path)

os.remove(input_path) 
os.rename(temp_path, input_path)

#duckdb.sql("SELECT COUNT(*) FROM query").show()
duckdb.sql(f"SELECT COUNT(*) FROM '{input_path}'").show()

file_weight = os.path.getsize(input_path)
print('.parquet size: ', round((file_weight/1024)/1024/1024, 2), 'GB')

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│     80082340 │
└──────────────┘

┌──────────┐
│ eliminar │
│  int64   │
├──────────┤
│    23740 │
└──────────┘

Cantidad de registros después de la limpieza:  80058600
┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│     80058600 │
└──────────────┘

.parquet size:  0.5 GB


#### Reescribimos bien los datos necesarios

In [33]:
duckdb.sql("""
    SELECT DISTINCT("year_of_fabrication") 
    FROM 'ignore_data_complete_clean.parquet'
    ORDER BY "year_of_fabrication" DESC
""").show(max_rows=100)

┌─────────────────────┐
│ year_of_fabrication │
│       varchar       │
├─────────────────────┤
│ 2023                │
│ 2022                │
│ 2021                │
│ 2020                │
│ 2019                │
│ 2018                │
│ 2017                │
│ 2016                │
│ 2015                │
│ 2014                │
│ 2013                │
│ 2012                │
│ 2011                │
│ 2010                │
├─────────────────────┤
│       14 rows       │
└─────────────────────┘



---

In [34]:
duckdb.sql("SELECT * FROM 'ignore_data_complete_clean.parquet' LIMIT 0").show()
duckdb.sql("""
           SELECT MAX("engine_size_L") FROM 'ignore_data_complete_clean.parquet'
""").show()

┌───────┬─────────────────────┬─────────┬──────────────┬──────────────────────────┬───────────────────┬─────────┬─────────────────┬───────────────────┬─────────┬───────────────────┬───────────┬──────────────────────────┬───────────────────┬───────────────┐
│  ID   │ year_of_fabrication │ country │ manufacturer │ energy_consumption_Wh/km │ electric_power_KW │  model  │ commercial_name │ electric_range_km │ mass_kg │ co2_emission_g/km │ fuel_type │ fuel_consumption_l/100km │ manufacturar_name │ engine_size_L │
│ int32 │       varchar       │ varchar │   varchar    │          int16           │       int16       │ varchar │     varchar     │       int16       │  int16  │       int16       │  varchar  │          float           │      varchar      │     float     │
├───────┴─────────────────────┴─────────┴──────────────┴──────────────────────────┴───────────────────┴─────────┴─────────────────┴───────────────────┴─────────┴───────────────────┴───────────┴──────────────────────────┴─────────

<p style="font-size: 25px; text-align: center; color: green;">Necesito un .parquet que no supere los 100MB para poder subirlo a GitHub. </p>

#### Convertir el archivo en html
<code>jupyter nbconvert --to html clean_data.ipynb</code>